In [34]:
import pandas as pd
import os
import datetime
import time
import random
import copy

In [2]:
pd.set_option("display.max_columns", None)

In [42]:
data = {}
param = {}
input_directory = '../temp'
param_directory = '../param'
for _filename in os.listdir(input_directory):
    if _filename[0] == '.':
        continue
    print(_filename)
    df_data = pd.read_csv(input_directory+'/'+_filename,  header = None)
    filetype = _filename.split('-')[1]
    column_dict = { 
            'sked': ['from', 'to', 'al','fln', 'actype', 'depday','dep','arrday','arr'],
            'comp': ['from', 'to', 'al', 'fln','actype', 'depday','dep','arrday','arr'],
            'demand': ['orig', 'dest', 'volume','weight', 'rev'],
            'demand_curve': ['orig', 'dest', 'ttt_1','ttt_2', 'ttt_3','ttt_4'],          
            'route_cost': ['from','to', 'actype', 'value'],
            'airport_cost': ['ap', 'actype', 'value'],
            'config': ['actype', 'cap_1', 'cap_2']
    }
    df_data.columns = column_dict[filetype]
    data[filetype] = df_data
    
for _filename in os.listdir(param_directory):
    if _filename[0] == '.':
        continue
    print(_filename)
    df_data = pd.read_csv(param_directory+'/'+_filename,  header = None)
    column_dict = { 
            'network' : ['stops_allowed'],
            'connections': ['hub', 'minct', 'maxct'],
            'preferences' : ['from', 'to', 'stop_penalty'],
            'config':['actype','cap','weight']
    }
    df_data.columns = column_dict[_filename]
    param[_filename] = df_data

test-airport_cost
test-comp
test-config
test-demand
test-demand_curve
test-route_cost
test-sked
config
connections
preferences


In [57]:
#inputs
your_sked = data['sked']
comp_sked = data['comp']
param_connect = param['connections']
param_pref = param['preferences']
param_pref['od'] = param_pref["from"]+param_pref["to"]
param_config = param['config']

max_stop = 2

your_sked = your_sked.merge(param_config, on='actype')
comp_sked = comp_sked.merge(param_config, on='actype')


full_sked = pd.concat([your_sked,comp_sked])
full_sked["dep"] = pd.to_datetime(full_sked["dep"]) + full_sked["depday"]*datetime.timedelta(days=1)
full_sked["arr"] = pd.to_datetime(full_sked["arr"]) + full_sked["arrday"]*datetime.timedelta(days=1)
full_sked["id"] = full_sked['al'] + full_sked['fln'].astype('str')+"/"+full_sked['depday'].astype('str')


param_connect['minct'] = pd.to_timedelta(param_connect['minct'])
param_connect['maxct'] = pd.to_timedelta(param_connect['maxct'])

df_connect = full_sked.add_suffix('_1').reset_index(drop=True)
df_connect['nbstops'] = 0

list_itin = {}
list_itin[0] = df_connect


#build itineraries
for i in range (1,max_stop+1):
    cnx_p = param_connect.copy()
    itins = list_itin[i-1].merge(full_sked.add_suffix('_'+str(i+1)), how= "cross")
    itins['nbstops'] = i
    
    itins['cnx_time_'+str(i)] = itins['dep_'+str(i+1)] - itins['arr_'+str(i)]
    
    cnx_p.columns = ['to_'+str(i), 'minct_'+str(i), 'maxct_'+str(i)]
    
    #filter connections
    itins.drop(itins[itins['to_'+str(i)] != itins['from_'+str(i+1)]].index, inplace=True)
    itins.drop(itins[itins['from_'+str(i)] == itins['to_'+str(i+1)]].index, inplace=True)
    itins.drop(itins[itins['from_'+str(1)] == itins['to_'+str(i+1)]].index, inplace=True)
    
    itins = itins.merge(cnx_p, on = ['to_'+str(i)])
    
    itins.drop(itins[itins['cnx_time_'+str(i)] < itins['minct_'+str(i)]].index, inplace=True)
    itins.drop(itins[itins['cnx_time_'+str(i)] > itins['maxct_'+str(i)]].index, inplace=True)
   
    list_itin[i] = itins


#add attributes
for i in range(0,max_stop+1):
    list_itin[i]["travel_time"] = list_itin[i]["arr_"+str(i+1)]-list_itin[i]["dep_1"]
    list_itin[i]["od"] = list_itin[i]["from_1"]+list_itin[i]["to_"+str(i+1)]
    list_itin[i]["index"] = list_itin[i].index
    list_itin[i]['id'] = list_itin[i]["nbstops"].astype(str)+"-"+list_itin[i]["index"].astype(str)
    list_itin[i] = list_itin[i].merge(param_pref, on="od")

    #calculate score
    list_itin[i]["score"]= list_itin[i]["travel_time"].dt.seconds+list_itin[i]["nbstops"]*list_itin[i]["stop_penalty"]*3600

    


In [58]:
def build_options(list_itin):
    od_itin = {}
    list_itin_summary = pd.DataFrame(columns=["id","index", "od","travel_time", "nbstops","score"])

    for i in range(0,max_stop+1):
        list_itin_summary= pd.concat([list_itin_summary,list_itin[i][["id","index", "od","travel_time", "nbstops","score"]]]) 
    #organize data by OD
    for od in param_pref.od.unique():    
        od_itin[od] = list_itin_summary[list_itin_summary['od']==od].copy()
    return list_itin_summary, od_itin

list_itin_summary, od_itin = build_options(list_itin)

In [59]:
#variables
timep = 4

#define user arrivals
demand_rand = {}
demand_curve = data['demand_curve']
demand = data['demand']
demand['od'] = demand['orig']+demand['dest']
demand_by_ttt = pd.merge(demand, demand_curve, on=["orig","dest"])

for i in range(1,1+timep):
    demand_by_ttt['d_ttt_'+str(i)]=(demand_by_ttt['ttt_'+str(i)]*demand_by_ttt['volume']).astype('int')

#shuffle pax

for i in range(1,1+timep):
    randm = []
    for indx, row in demand_by_ttt.iterrows():
        for k in range(1,row['d_ttt_'+str(i)]):
            randm.append(row['orig']+row['dest'])
    random.shuffle(randm)
    demand_rand[i] = randm
    

In [60]:
spill = dict.fromkeys(demand['od'].unique(),0)

avail_list_itin = {}
for key in list_itin:   
    avail_list_itin[key] = list_itin[key].copy()

avail_od_itin = {}
for key in od_itin:
    avail_od_itin[key] = od_itin[key].copy()

avail_list_itin_summary = list_itin_summary.copy()

list_pax = demand_rand[1]


#choose an itin randomly among available ones
for od_pax in list_pax:
    if len(avail_od_itin[od_pax])>0 :
        choice = random.choices(list(avail_od_itin[od_pax]["id"]), weights = list(avail_od_itin[od_pax]["score"]), k=1)
        stop_index = choice[0].split('-')
        choice_stops = int(stop_index[0])
        choice_index = int(stop_index[1])  
        itin_selected = avail_list_itin[choice_stops]['index']==choice_index
        #add pax and revenue to itinerary 
        #add pax and prorated revenue to flight

        #remove capacity from all itineraries with the flights
        for i in range(0,choice_stops+1):
            avail_list_itin[choice_stops].loc[itin_selected,'cap_'+str(i+1)] -= 1

            if avail_list_itin[choice_stops].loc[itin_selected, 'cap_'+str(i+1)].size>0 and avail_list_itin[choice_stops].loc[itin_selected, 'cap_'+str(i+1)].values==0:
              
                closed_flight = avail_list_itin[choice_stops].loc[itin_selected,'id_'+str(i+1)]
                 #remove all itineraries with the flights that is overbooked
                for j in range(0, max_stop+1):
                    avail_list_itin[j].drop(avail_list_itin[j][avail_list_itin[j]['id_'+str(j+1)] == closed_flight.values[0]].index, inplace=True)
                    avail_list_itin_summary, avail_od_itin = build_options(avail_list_itin)


    else:
        #log spill
        spill[od_pax] += 1 


In [61]:
spill

{'DUSTXL': 57,
 'DUSCDG': 168,
 'DUSFRA': 34,
 'DUSLHR': 520,
 'DUSMAD': 76,
 'DUSBCN': 126,
 'DUSFCO': 126,
 'TXLMAD': 18,
 'CDGMAD': 16,
 'FRAMAD': 69,
 'LHRMAD': 19,
 'BCNMAD': 22,
 'FCOMAD': 0}

In [ ]:
list_itin[0]

In [62]:
avail_list_itin[0]

,from_1,to_1,al_1,fln_1,actype_1,depday_1,dep_1,arrday_1,arr_1,cap_1,weight_1,id_1,nbstops,travel_time,od,index,id,from,to,stop_penalty,score
15,FCO,MAD,AZ,37,737,4,2022-02-05 05:35:00,4,2022-02-05 06:43:00,25,57000,AZ37/4,0,0 days 01:08:00,FCOMAD,38,0-38,FCO,MAD,1,4080


In [ ]:
od_itin['DUSTXL']

In [ ]:
avail_od_itin['DUSTXL']